## Neural Network Cross Val - MLPClassifier

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import confusion_matrix
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV

In [2]:
import dill
dill.load_session('notebook_env.db')

In [13]:
import dill
dill.dump_session('notebook_env2.db')

In [4]:
def detector_cross_val(classifier, XTrain, yTrain, folds, score = "f1_score"):
    
    yTrain = np.array(yTrain)
    if type(XTrain) == type(pd.DataFrame()):
        XTrain = XTrain.as_matrix()
    #StratifiedKFold
    k_fold = StratifiedKFold(folds, random_state= 123, shuffle=True)
    fold_scores = []
    for k, (train, val) in enumerate(k_fold.split(XTrain, yTrain)):
        classifier.fit(XTrain[train],yTrain[train])
        y_pred = classifier.predict(XTrain[val])
        y_val = yTrain[val]
        if score == "f1_score":
            fold_scores.append(f1_score(y_val, y_pred, average='weighted'))
        else:
            fold_scores.append(accuracy_score(y_val, y_pred))
    
    return np.mean(fold_scores)

In [7]:
NN_score = []
lr = ["invscaling", "adaptive"]
hl = [(100,30), (100,10)]
alpha = np.logspace(-2,4,3)
#act = ["logistic", "relu", "Tanh"]
for i in lr:
    for j in hl:
        for k in alpha:
            NN_score.append([i, j, k, detector_cross_val(MLPClassifier(hidden_layer_sizes=j, alpha = k,  learning_rate= i), X_train, y_train, folds= 3)])
            print(i,j, end= " ")

invscaling (100, 30) invscaling (100, 30) 

C:\Users\dmcph\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


invscaling (100, 30) invscaling (100, 10) invscaling (100, 10) invscaling (100, 10) adaptive (100, 30) adaptive (100, 30) adaptive (100, 30) adaptive (100, 10) adaptive (100, 10) adaptive (100, 10) 

In [9]:
NN_score.sort(reverse=True, key = lambda x: x[3])
NN_score

[['adaptive', (100, 30), 0.01, 0.91230916671973683],
 ['invscaling', (100, 30), 0.01, 0.90875682576409755],
 ['adaptive', (100, 10), 0.01, 0.90850921231595516],
 ['invscaling', (100, 10), 0.01, 0.90723408569066699],
 ['invscaling', (100, 10), 10.0, 0.68105838393171958],
 ['adaptive', (100, 30), 10.0, 0.67773210626198666],
 ['adaptive', (100, 10), 10.0, 0.67509092591017528],
 ['invscaling', (100, 30), 10.0, 0.67459611345230719],
 ['invscaling', (100, 30), 10000.0, 0.33333333333333331],
 ['invscaling', (100, 10), 10000.0, 0.33333333333333331],
 ['adaptive', (100, 30), 10000.0, 0.33333333333333331],
 ['adaptive', (100, 10), 10000.0, 0.33333333333333331]]

In [12]:
clf = MLPClassifier(hidden_layer_sizes=(100,30), alpha = .001,  learning_rate= 'adaptive').fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(f1_score(y_test, y_pred, average='weighted'))
cm = confusion_matrix(y_test, y_pred)
cm

0.909986455161


array([[98313, 11140],
       [ 1586,  5798]], dtype=int64)